# MLflow Tracking


# Demonstração de MLflow
Este notebook demonstra como utilizar o **MLflow** para rastrear experimentos de machine learning, registrar parâmetros, métricas, artefatos e gerenciar modelos registrados. A seguir estão os principais tópicos abordados:

1. **Rastreamento de Experimentos**: Rastreando os parâmetros, métricas e artefatos de modelos treinados.
2. **Logging Automático**: Automatizando o processo de registro no MLflow.


# Imports

In [0]:
import mlflow
import mlflow.sklearn
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
import pdb


In [0]:
experiment_id = "100091573071916"  # ID do experimento
mlflow.set_experiment(experiment_id=experiment_id)

# Carregamento de dados

In [0]:
# Carregar dataset
data = load_iris()
X, y = data.data, data.target

# Dividir dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# MLflow Tracking

In [0]:

# Inicializar execução do mlflow
with mlflow.start_run():

    pdb.set_trace()  # Pausar utilizando pdb

    # Definir hiperparâmetros
    n_estimators = 100
    max_depth = 5

    # Registrar parâmetros
    mlflow.log_params({
        "n_estimators": n_estimators,
        "max_depth": max_depth,
        "dataset_size": len(X)
    })

    # Treinar modelo
    rf_classifier = RandomForestClassifier(
        n_estimators=n_estimators, 
        max_depth=max_depth,
        random_state=42
    )
    rf_classifier.fit(X_train, y_train)
    
    # Fazer previsões
    y_pred = rf_classifier.predict(X_test)

    # Registrar métricas
    accuracy = accuracy_score(y_test, y_pred)
    mlflow.log_metrics({
        "accuracy": accuracy,
        "test_size": len(X_test)
    })

    pdb.set_trace()  # Pausar pdb

    # Criar e registrar visualização da importância das características
    plt.figure(figsize=(10, 6))
    feature_importance = pd.DataFrame({
        'feature': data.feature_names,
        'importance': rf_classifier.feature_importances_
    }).sort_values('importance', ascending=False)

    top_features = feature_importance.head(10)
    plt.barh(top_features['feature'], top_features['importance'])
    plt.title('Top Feature Importances')
    plt.xlabel('Importance')
    plt.tight_layout()
    
    # Salvar e registrar o gráfico de importância das características
    plt.savefig('/tmp/feature_importance_iris.png')
    mlflow.log_artifact('/tmp/feature_importance_iris.png')

    # Criar e registrar visualização da matriz de confusão
    plt.figure(figsize=(8, 6))
    cm = confusion_matrix(y_test, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=data.target_names, yticklabels=data.target_names)
    plt.title('Iris Classification Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    
    # Salvar e registrar a matriz de confusão como um artefato
    plt.savefig('/tmp/confusion_matrix_iris.png')
    mlflow.log_artifact('/tmp/confusion_matrix_iris.png')

    pdb.set_trace()
    
    # Registrar o modelo
    mlflow.sklearn.log_model(
        rf_classifier, 
        "random_forest_model",
        signature=mlflow.models.signature.infer_signature(X_train, y_train)
    )



## Seção 2: Auto-logging
Nesta seção, utilizamos o recurso de **auto-logging** do MLflow para registrar automaticamente todos os parâmetros, métricas e artefatos de um modelo. O MLflow detecta automaticamente as informações relevantes e as registra no experimento configurado.


In [0]:
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

# Enable auto logging
mlflow.sklearn.autolog()

# Train and log model with auto logging
with mlflow.start_run():
    model = RandomForestClassifier(n_estimators=150, max_depth=6, random_state=42)
    model.fit(X_train, y_train)
